In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> imports >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
!pip install inflect
!pip install contractions 

import contractions
from contractionsOne import CONTRACTION_MAP

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')


import inflect
import numpy as np
import pandas as pd
import sys, setuptools, tokenize
import re, unicodedata, string  
from bs4 import BeautifulSoup

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loading the dataset >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
pd.options.mode.chained_assignment = None
# df1 = pd.read_csv('/content/Headphone extracted droped columns.csv')
# product_reviews_hp_droped = df1.drop(['brand','overall','vote','verified','summary', 'reviewerID', 'unixReviewTime', 'asin', 'title'], axis="columns")

df = pd.read_csv('/content/testing.csv')
df

,reviewText
0,<h1>I've been using these for over a year now....
1,NaN
2,https://www.hi.com I have owned these for over...
3,I'll be there within 5 min. Shouldn't you be t...
4,I ** [bought] these thinking they were the tic...
5,#Work Good. I do not have another reference. Y...
6,भारत Not as immersive and noise-cancelling as ...
7,DINUVI I own these and the Bose Quiet Comfort...
8,�� I bought these at the flea market for 10 do...
9,Let's make it short story. It extremly difficu...


In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing null values >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#removing null values
# counting rows after removing null values
df.dropna(subset=['reviewText'], inplace=True)
df

,reviewText
0,<h1>I've been using these for over a year now....
2,https://www.hi.com I have owned these for over...
3,I'll be there within 5 min. Shouldn't you be t...
4,I ** [bought] these thinking they were the tic...
5,#Work Good. I do not have another reference. Y...
6,भारत Not as immersive and noise-cancelling as ...
7,DINUVI I own these and the Bose Quiet Comfort...
8,�� I bought these at the flea market for 10 do...
9,Let's make it short story. It extremly difficu...
10,I purchased this product because I could never...


In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing URL >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def remove_URL(reviewText):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', str(reviewText))
df['reviewText']=df['reviewText'].apply(remove_URL)
df.head(10)

,reviewText
0,<h1>I've been using these for over a year now....
2,I have owned these for over two years and wor...
3,I'll be there within 5 min. Shouldn't you be t...
4,I ** [bought] these thinking they were the tic...
5,#Work Good. I do not have another reference. Y...
6,भारत Not as immersive and noise-cancelling as ...
7,DINUVI I own these and the Bose Quiet Comfort...
8,�� I bought these at the flea market for 10 do...
9,Let's make it short story. It extremly difficu...
10,I purchased this product because I could never...


In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing HTML tags >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def remove_html(reviewText):
    html=re.compile(r'<.*?>')
    return html.sub(r'',str(reviewText))
df['reviewText'] = df['reviewText'].apply(remove_html)
df.head(10)

,reviewText
0,I've been using these for over a year now.
2,I have owned these for over two years and wor...
3,I'll be there within 5 min. Shouldn't you be t...
4,I ** [bought] these thinking they were the tic...
5,#Work Good. I do not have another reference. Y...
6,भारत Not as immersive and noise-cancelling as ...
7,DINUVI I own these and the Bose Quiet Comfort...
8,�� I bought these at the flea market for 10 do...
9,Let's make it short story. It extremly difficu...
10,I purchased this product because I could never...


In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing square brackets and the inside of the square bracket >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def remove_between_square_brackets(reviewText):
    return re.sub('\[[^]]*\]', '', str(reviewText))
df['reviewText'] = df['reviewText'].apply(remove_between_square_brackets)
df.head(10)

,reviewText
0,I've been using these for over a year now.
2,I have owned these for over two years and wor...
3,I'll be there within 5 min. Shouldn't you be t...
4,I ** these thinking they were the ticket to n...
5,#Work Good. I do not have another reference. Y...
6,भारत Not as immersive and noise-cancelling as ...
7,DINUVI I own these and the Bose Quiet Comfort...
8,�� I bought these at the flea market for 10 do...
9,Let's make it short story. It extremly difficu...
10,I purchased this product because I could never...


In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing Pictures/Tags/Symbols/Emojis >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def remove_emoji(reviewText):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', str(reviewText))

df['reviewText'] = df['reviewText'].apply(remove_emoji)
df.head(10)

,reviewText
0,I've been using these for over a year now.
2,I have owned these for over two years and wor...
3,I'll be there within 5 min. Shouldn't you be t...
4,I ** these thinking they were the ticket to n...
5,#Work Good. I do not have another reference. Y...
6,भारत Not as immersive and noise-cancelling as ...
7,DINUVI I own these and the Bose Quiet Comfort...
8,I bought these at the flea market for 10 doll...
9,Let's make it short story. It extremly difficu...
10,I purchased this product because I could never...


In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing non ascii >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
df['reviewText'] = [w.encode("ascii", "ignore").decode() for w in df['reviewText']]
df

,reviewText
0,I've been using these for over a year now.
2,I have owned these for over two years and wor...
3,I'll be there within 5 min. Shouldn't you be t...
4,I ** these thinking they were the ticket to n...
5,#Work Good. I do not have another reference. Y...
6,Not as immersive and noise-cancelling as othe...
7,DINUVI I own these and the Bose Quiet Comfort...
8,I bought these at the flea market for 10 doll...
9,Let's make it short story. It extremly difficu...
10,I purchased this product because I could never...


In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> convert to lower case >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
df['reviewText'] = [w.lower() for w in df['reviewText']]
df

,reviewText
0,i've been using these for over a year now.
2,i have owned these for over two years and wor...
3,i'll be there within 5 min. shouldn't you be t...
4,i ** these thinking they were the ticket to n...
5,#work good. i do not have another reference. y...
6,not as immersive and noise-cancelling as othe...
7,dinuvi i own these and the bose quiet comfort...
8,i bought these at the flea market for 10 doll...
9,let's make it short story. it extremly difficu...
10,i purchased this product because i could never...


In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> expand contractions >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def expand_contractions(reviewText):
    contractionsPattern = re.compile('({})'.format('|'.join(CONTRACTION_MAP.keys())),flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        firstChar = match[0]
        expandedContraction = CONTRACTION_MAP.get(match) \
            if CONTRACTION_MAP.get(match) \
            else CONTRACTION_MAP.get(match.lower())
        expandedContraction = firstChar+expandedContraction[1:]
        return expandedContraction
    reviewText = contractionsPattern.sub(expand_match, str(reviewText))
    reviewText = re.sub("'", "", reviewText)
    return reviewText

df['reviewText']=df['reviewText'].apply(expand_contractions)
df

,reviewText
0,i have been using these for over a year now.
2,i have owned these for over two years and wor...
3,i will be there within 5 min. should not you b...
4,i ** these thinking they were the ticket to n...
5,#work good. i do not have another reference. y...
6,not as immersive and noise-cancelling as othe...
7,dinuvi i own these and the bose quiet comfort...
8,i bought these at the flea market for 10 doll...
9,let us make it short story. it extremly diffic...
10,i purchased this product because i could never...


In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> tokenizing sentence >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
sentence_all = []
for index, row in df.iterrows():
  sentences_with_feature = []
  for sen in sent_tokenize(row['reviewText']):
    sentences_with_feature.append(sen)
  sentence_all.append(sentences_with_feature)

list_of_tuples = list(zip(sentence_all)) 
df_sentences = pd.DataFrame(list_of_tuples,columns = ['reviewText'])
df_sentences

,reviewText
0,[i have been using these for over a year now.]
1,[ i have owned these for over two years and wo...
2,"[i will be there within 5 min., should not you..."
3,[i ** these thinking they were the ticket to ...
4,"[#work good., i do not have another reference...."
5,[ not as immersive and noise-cancelling as oth...
6,[dinuvi i own these and the bose quiet comfor...
7,[ i bought these at the flea market for 10 dol...
8,"[let us make it short story., it extremly diff..."
9,[i purchased this product because i could neve...


In [ ]:
for index, row in df_sentences.iterrows():

  list_of_tuples = list(zip(df_sentences)) 
  df_sentences_word = pd.DataFrame(list_of_tuples,columns = ['reviewText'])

  # wordTokenize(row['reviewText'])
df_sentences_word

,reviewText
0,reviewText


In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> tokenizing words >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def wordTokenize(sentence):
  tokens = [w for t in (sentence.apply(word_tokenize)) for w in t]
  return tokens

In [ ]:
s

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> replacing numbers with string  >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def stringNumbers(sentence):
  p = inflect.engine()
  stringNumbers = []
  for word in removedPunctuation:
      if word.isdigit():
  #         new_word = num2word.to_card(15)
          new_word = p.number_to_words(word)
          stringNumbers.append(new_word)
      else:
          stringNumbers.append(word)
  # stringNumbers = [w.lower() for w in removedPunctuation]
  return stringNumbers

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing stopwords >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def stopwords(sentence):
  stop_words = set(stopwords.words('english'))
  removedStopwords = [w for w in stringNumbers if not w in stop_words]
  return removedStopwords

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> stemming words >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def stemmer(sentence):
  stemmer = PorterStemmer()
  # stemmer = LancasterStemmer()
  stems = []
  for word in removedStopwords:
      stem = stemmer.stem(word)
      stems.append(stem)
  return stems

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> lemmatize_verbs >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def lemmatizer():
  lemmatizer = WordNetLemmatizer()
  lemmas = []
  for word in stems:
      lemma = lemmatizer.lemmatize(word, pos='v')
      lemmas.append(lemma)
  return lemmas

In [ ]:
# saving the dataframe 
dataFrame = pd.DataFrame(lemmas) 

list_of_tuples = list(zip(lemmas)) 
df_audio_sentences = pd.DataFrame(list_of_tuples,columns = ['review'])

df_audio_sentences.to_csv('testing_cleaned_3.csv') 
df_audio_sentences